# 1. Activate GPU and Install Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

False

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.2 MB/s 
     |████████████████████████████████| 5.5 MB 41.1 MB/s 
     |████████████████████████████████| 163 kB 47.1 MB/s 
     |████████████████████████████████| 115 kB 57.8 MB/s 
     |████████████████████████████████| 212 kB 59.3 MB/s 
     |████████████████████████████████| 95 kB 6.2 MB/s 
     |████████████████████████████████| 127 kB 73.1 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 
     |████████████████████████████████| 115 kB 69.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


#2. Preprocess data

In [ ]:
# Load data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/twitter/Twitter Preprocessed/profane_preprocessed_final.csv')
# df = df.dropna()
from imblearn.over_sampling import SMOTE
from collections import Counter

x, y = [], []
d=dict()
reverse=dict()
c=0
for i in range (len(df)):
  t = df.loc[i,'text']
  if t not in d:
    d[t]=c
    reverse[c]=t
    c+=1
  x.append([d[t]])
  y.append(df.loc[i,'label'])

oversample = SMOTE()
X, Y = oversample.fit_resample(x, y)

print(Counter(Y))
x1 = []
for i in X:
  x1.append(reverse[i[0]])

df = pd.DataFrame(list(zip(x1, Y)), columns =['text', 'label'])
df = df.dropna()
df.head()

Counter({0: 3019, 1: 3019})


,text,label
0,RT : We want Justice.... We want Vacancy... Wa...,0
1,RT : भारत की धरोहर को जब विश्व के लोग अपना रहे...,0
2,RT : UP_POLICE_VACANCY\n …,0
3,"RT : माननीय प्रधानमंत्री जी, कृपया कर पूरे भार...",0
4,😅😅😅😅😅😅 joke?,0


In [ ]:
2# Create a smaller training dataset for faster training times
from sklearn.model_selection import train_test_split
small_train_dataset, small_test_dataset = train_test_split(df, test_size=0.15, random_state=62)
#small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
#small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(len(small_train_dataset))
print(len(small_test_dataset))

5131
906


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

dataset = ds.dataset(pa.Table.from_pandas(small_train_dataset).to_batches())

### convert to Huggingface dataset
train = Dataset(pa.Table.from_pandas(small_train_dataset))

dataset = ds.dataset(pa.Table.from_pandas(small_test_dataset).to_batches())

### convert to Huggingface dataset
test = Dataset(pa.Table.from_pandas(small_test_dataset))

print(type(train), type(test), train[0])


<class 'datasets.arrow_dataset.Dataset'> <class 'datasets.arrow_dataset.Dataset'> {'text': 'You are welcome ??', 'label': 0, '__index_level_0__': 1857}


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-electra-base")

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
  return tokenizer(examples["text"], max_length=512, truncation=True)

tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(tokenized_train)

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5131
})


In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
print(data_collator)

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='cross-encoder/ms-marco-electra-base', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')


# 3. Training the model

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-electra-base", num_labels=2, ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at cross-encoder/ms-marco-electra-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([1, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([1]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the evaluation metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    load_precision = load_metric("precision")
    lead_recall = load_metric("recall")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"] #(predictions=predictions, references=labels, average='micro')
    precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
    recall = lead_recall.compute(predictions=predictions, references=labels)["recall"]

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
# Define a new Trainer with all the objects we constructed so far
#finetuned_deberta_profane
from transformers import TrainingArguments, Trainer

repo_name = "finetune_profane_improved_v1"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/SayakRana/finetune_profane_improved_v1 into local empty directory.


In [ ]:
# Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5131
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3210
  Number of trainable parameters = 109483778
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using th

Step,Training Loss


In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

# 4. Analyzing new data with the model

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="SayakRana/finetune_profane_improved_v1")

In [ ]:
sentiment_model(["Rishi Sunak is now the first Hindu PM of Britain", "India lost the world cup match"])